#### Code for NWchem-Qiskit Driver
##### 1. import package

In [42]:
import os 
import sys
import yaml
import os
from qiskit_nature_qe import nwchem_driver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.exceptions import QiskitError
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD, UCC
import numpy as np
from nwchem2yaml import extract_fields

In [43]:
# data_dir = os.path.join("qe_files", "n2", "output")
# datafile = os.path.join(data_dir, "demo.out")
# data_file_yaml = os.path.join(data_dir, "demo.yaml")

data_dir = os.path.join("qe_files", "3x3_periodic", "NWChem",'1covo')
datafile = os.path.join(data_dir, "H1Li1-7.0.out")
data_file_yaml = os.path.join(data_dir, "H1Li1-7.0.yaml")

##### 2. nwchem output to yaml

In [44]:
data = extract_fields(datafile)
with open(data_file_yaml, 'w') as f:
    f.write(yaml.dump(data, default_flow_style=False)) 

##### 3. data to Qiskit electron structure problem class

In [45]:
driver = nwchem_driver.NWchem_Driver(data_file_yaml)
es_problem = driver.run()
# hamiltonian  = es_problem.hamiltonian
# print(hamiltonian.second_q_op())

h_ij up-down equal: True
eri up-down equal: True
eri up-(down-up) equal: False
eri (up-down)-(down-up) equal: True


##### 4. Mapping Hamiltonian to qubit

In [46]:
mapper = JordanWignerMapper()

##### 5. specify ansatz

In [47]:
ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
        ),
    )

##### 6. Constructing VQE and check results 

In [48]:
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
# vqe_solver.initial_point = [0.0] * ansatz.num_parameters
vqe_solver.initial_point = np.random.rand(ansatz.num_parameters)
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)
print('> Total energy from HF (Hartree):{}'.format(driver.total_energy))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -0.60728322838
  - computed part:      -0.60728322838
~ Nuclear repulsion energy (Hartree): -0.1028054491
> Total ground state energy (Hartree): -0.71008867748
> Total energy from HF (Hartree):-0.591167016
